<a href="https://colab.research.google.com/github/bananighosh/llama3-from-scratch/blob/main/llama3_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!bash download.sh

Enter the URL from email: https://download6.llamameta.net/*?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoiZGE1NHIycHhxM2wwZWswdGd3dnBuZ2lkIiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZG93bmxvYWQ2LmxsYW1hbWV0YS5uZXRcLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3MTY4NzAxNzV9fX1dfQ__&Signature=k3ePhF85Fk12gM92SOGEY6KHSR3p64C1FksFz5FiYrWSw7FGgdgbeRiQUNET3wR-bpXQpZxczG%7EOUkW6hN0oybYFPDVhOuic48NjJmKrwlmwE1Q1eJzsFDatmn%7E80viS%7E4viK0sD0Xc7wPPT1HatC4ovrCQP9wO7qXrJdjB82tsIXKakx%7Eb-bsXQTvTixSMh3jU-kbDpximA%7E91CnRjnkzwhRPclGWx6k8dy7qjikgOdqVDFzl-uSZa85xWpv7KkeIH2GfL4hgEOFj15lpfk5O4FP99lM-Ysda7dfDsTBhXukGHlnTLD0ftHNRD8dHq7FaagzVyiJhlNNhTAurEh3A__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=337416229137869

Enter the list of models to download without spaces (8B,8B-instruct,70B,70B-instruct), or press Enter for all: 8B,8B-instruct
--2024-05-27 06:37:50--  https://download6.llamameta.net/LICENSE?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoiZGE1NHIycHhxM2wwZWswdGd3dnBuZ2lkIiwiUmVzb

In [23]:
# !huggingface-cli download meta-llama/Meta-Llama-3-8B-Instruct --include "original/*" --local-dir meta-llama/Meta-Llama-3-8B-Instruct

In [24]:
# import transformers
# import torch

# model_id = "meta-llama/Meta-Llama-3-8B"

# pipeline = transformers.pipeline(
#   "text-generation",
#   model="meta-llama/Meta-Llama-3-8B",
#   model_kwargs={"torch_dtype": torch.bfloat16},
#   device="cuda",
# )

# Tokenizer

In [2]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.5 MB/s eta 0:00:00


In [3]:
!pip install blobfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.1 MB/s eta 0:00:00


In [4]:
from pathlib import Path
import tiktoken
from tiktoken.load import load_tiktoken_bpe
import torch
import json
import matplotlib.pyplot as plt

In [8]:
tokenizer_path = "Meta-Llama-3-8B-Instruct/tokenizer.model"

special_tokens = [
            "<|begin_of_text|>",
            "<|end_of_text|>",
            "<|reserved_special_token_0|>",
            "<|reserved_special_token_1|>",
            "<|reserved_special_token_2|>",
            "<|reserved_special_token_3|>",
            "<|start_header_id|>",
            "<|end_header_id|>",
            "<|reserved_special_token_4|>",
            "<|eot_id|>",  # end of turn
        ] + [f"<|reserved_special_token_{i}|>" for i in range(5, 256 - 5)]

mergeable_ranks = load_tiktoken_bpe(tokenizer_path)

tokenizer = tiktoken.Encoding(
    name=Path(tokenizer_path).name,
    pat_str=r"(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+",
    mergeable_ranks=mergeable_ranks,
    special_tokens={token: len(mergeable_ranks) + i for i, token in enumerate(special_tokens)},
)

tokenizer.decode(tokenizer.encode("Hello Banani !"))

'Hello Banani !'

# analysing the model

In [ ]:
model = torch.load("Meta-Llama-3-8B-Instruct/consolidated.00.pth")

In [ ]:
print(json.dumps(list(model.keys())[:20], indent=4))

In [9]:
with open("Meta-Llama-3-8B-Instruct/params.json", "r") as f:
    config = json.load(f)
config

{'dim': 4096,
 'n_layers': 32,
 'n_heads': 32,
 'n_kv_heads': 8,
 'vocab_size': 128256,
 'multiple_of': 1024,
 'ffn_dim_multiplier': 1.3,
 'norm_eps': 1e-05,
 'rope_theta': 500000.0}

In [18]:
dim = config["dim"]
n_layers = config["n_layers"]
n_heads = config["n_heads"]
n_kv_heads = config["n_kv_heads"]
vocab_size = config["vocab_size"]
multiple_of = config["multiple_of"]
ffn_dim_multiplier = config["ffn_dim_multiplier"]
norm_eps = config["norm_eps"]
rope_theta = torch.tensor(config["rope_theta"])

# text to tokens

In [13]:
prompt = "the answer to the ultimate question of life, the universe, and everything is "
tokens = [128000] + tokenizer.encode(prompt)
print(tokens)

[128000, 1820, 4320, 311, 279, 17139, 3488, 315, 2324, 11, 279, 15861, 11, 323, 4395, 374, 220]


In [17]:
tokens = torch.tensor(tokens)
prompt_split_as_tokens = [tokenizer.decode([token.item()]) for token in tokens]
for token in prompt_split_as_tokens:
  print(f"'{token}',")
# print(prompt_split_as_tokens)

'<|begin_of_text|>',
'the',
' answer',
' to',
' the',
' ultimate',
' question',
' of',
' life',
',',
' the',
' universe',
',',
' and',
' everything',
' is',
' ',


<ipython-input-17-d7f19a8679ad>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens = torch.tensor(tokens)


# tokens to embeddings

In [21]:
embedding_layer = torch.nn.Embedding(vocab_size, dim)
print(embedding_layer)

Embedding(128256, 4096)
